In [1]:
import os
import pymongo
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer, MinMaxScaler
from joblib import Parallel, delayed, Memory

In [ ]:
print(librosa.__version__)

In [ ]:
tracks = pd.read_csv('fma_metadata\\tracks.csv')
genres = pd.read_csv('fma_metadata\\genres.csv')
features = pd.read_csv('fma_metadata\\features.csv')
echonest = pd.read_csv('fma_metadata\\echonest.csv')
raw_albums = pd.read_csv('fma_metadata\\raw_albums.csv')
raw_artists = pd.read_csv('fma_metadata\\raw_artists.csv')
raw_genres = pd.read_csv('fma_metadata\\raw_genres.csv')
raw_tracks = pd.read_csv('fma_metadata\\raw_tracks.csv')
raw_echonest = pd.read_csv('fma_metadata\\raw_echonest.csv')

In [ ]:
# finding a specifc artist in the raw_artist by using the name and if not found display a message
def find_artist(artist_name):
    # convert the artist_name column to lower case
    artist_name = artist_name.lower()
    raw_artists['artist_name'] = raw_artists['artist_name'].str.lower()
    
    # check if the artist_name is in the raw_artists dataframe
    if artist_name.lower() in raw_artists['artist_name'].values:
        print('Artist found')
    else:
        print('Artist not found')
        
        
artist_name = 'lucky dragons'
find_artist(artist_name)

In [3]:
import os
import numpy as np
import librosa
from joblib import Parallel, delayed, Memory
from soundfile import SoundFile

# Path to the directory containing audio files
directory_path = 'D:\\fma_small'

# Initialize memory cache
memory = Memory(location='./cache', verbose=0)

# Initialize lists to store features and corresponding labels
mfcc_features_list = []
labels = []

# Define a function to extract features from an audio file
@memory.cache
def extract_features(audio_file_path):
    try:
        with SoundFile(audio_file_path) as audio_file:
            audio_signal = audio_file.read(dtype='float32')
            sampling_rate = audio_file.samplerate
        mfcc_features = librosa.feature.mfcc(y=audio_signal, sr=sampling_rate, n_mfcc=13)
        spectral_centroid = librosa.feature.spectral_centroid(y=audio_signal, sr=sampling_rate)
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio_signal)
        return mfcc_features.T, spectral_centroid.squeeze(), zero_crossing_rate.squeeze()
    except Exception as e:
        print(f"Error processing file {audio_file_path}: {e}")
        return None, None, None

# Iterate through each folder (genre) in the directory
for folder_name in os.listdir(directory_path):
    genre_folder_path = os.path.join(directory_path, folder_name)

    try:
        # Use joblib to parallelize feature extraction across audio files
        results = Parallel(n_jobs=-1)(
            delayed(extract_features)(os.path.join(genre_folder_path, audio_file_name))
            for audio_file_name in os.listdir(genre_folder_path)
        )

        # Unpack results and append features and labels
        for mfcc_features, spectral_centroid, zero_crossing_rate in results:
            if mfcc_features is not None:
                mfcc_features_list.append(mfcc_features)
                labels.append(folder_name)
    except Exception as e:
        print(f"Error processing folder {genre_folder_path}: {e}")

# Concatenate MFCC features to create a single array
mfcc_features_array = np.concatenate(mfcc_features_list)

# Display the shape of the MFCC features array and labels
print(mfcc_features_array.shape)

In [ ]:
# RobustScaler for standardization
robust_scaler = MinMaxScaler()
robust_standardized_mfcc_features = robust_scaler.fit_transform(mfcc_features)

# # Normalizer the stndardized features
normalizer = Normalizer("l2")
normalized_spectral_centroid = normalizer.fit_transform(spectral_centroid.reshape(1, -1))
normalized_zero_crossing_rate = normalizer.fit_transform(zero_crossing_rate.reshape(1, -1))

# Display the first 5 rows of the normalized MFCC features
print("Standardized MFCC Features: ",robust_standardized_mfcc_features[:5])
print("________________________")
print("Normalized Spectral Centrioid: ",normalized_spectral_centroid[:5])
print("________________________")
print("Normalized Zero Crossing Rate: ",normalized_zero_crossing_rate[:5])

In [ ]:
# applying pca on the features
from sklearn.decomposition import PCA

# Initialize PCA with 2 components
pca = PCA(n_components=2)

# Fit and transform the standardized MFCC features
pca_mfcc_features = pca.fit_transform(robust_standardized_mfcc_features)

# Display the shape of the PCA features
print(pca_mfcc_features)

In [ ]:
# save the normalized features to a npy
np.save('normalized_mfcc_features.npy', robust_standardized_mfcc_features)
np.save('normalized_spectral_centroid.npy', normalized_spectral_centroid)
np.save('normalized_zero_crossing_rate.npy', normalized_zero_crossing_rate)
np.save('labels.npy', labels)

In [ ]:
# saving the features to a mongoDB 
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["music_features"]
mfcc_collection = db["mfcc_features"]
spectral_centroid_collection = db["spectral_centroid"]
zero_crossing_rate_collection = db["zero_crossing_rate"]

# Insert the features into the collection
# convert the features to a list
# Convert the features to a list of dictionaries
robust_standardized_mfcc_features = [ {"value": x} for x in robust_standardized_mfcc_features ]
normalized_spectral_centroid = [ {"value": x} for x in normalized_spectral_centroid ]
normalized_zero_crossing_rate = [ {"value": x} for x in normalized_zero_crossing_rate ]

# Insert the features into the collection
mfcc_collection.insert_many(robust_standardized_mfcc_features)
spectral_centroid_collection.insert_many(normalized_spectral_centroid)
zero_crossing_rate_collection.insert_many(normalized_zero_crossing_rate)


In [ ]:
# Display the first document in the collection
print(mfcc_collection.find_one())
print(spectral_centroid_collection.find_one())
print(zero_crossing_rate_collection.find_one())